In [257]:
import adagram
import gensim
import pandas as pd
from lxml import html
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from string import punctuation
from collections import Counter
import numpy as np
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
morph = MorphAnalyzer()
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words]

    return ' '.join(words)

def tokenize(text):
    
    words = [word.strip(punct) for word in text.lower().split() if word and word not in stops]

    return ' '.join(words)

## Адаграм

Word2Vec и Fasttext каждому слову (или буквенному нграму) сопоставляют 1 вектор. Это значит, что у каждого слова в векторном пространстве только 1 значение. У многозначных слов векторы будут просто каким-то усреднением или обобщением всех его значений. 

В работе https://arxiv.org/pdf/1502.07257.pdf предлагается способ улучшить Skip Gram, так чтобы каждому слову сопостовлялось K различных векторов, так что каждый из них представляет какое-то из его значений. При этом сам параметр K задавать не нужно, модель сама находит нужное количество "значений" для каждого слова.

Изначально этот  подход реализован на julia, но есть реализация на питоне - https://github.com/lopuhin/python-adagram

Обучается адаграм через командную строку.

Возьмем данные с прошлого семинара и сохраним их в один файл.

In [4]:
data_rt = pd.read_csv('news_texts.csv')

In [6]:
data_rt.dropna(inplace=True)

In [7]:
corpus = ' '.join(data_rt.content_norm)

In [9]:
f = open('corpus.txt', 'w')
f.write(corpus)
f.close()

Обучим модель (обучается долго поэтому можете пропустить этот шаг)

In [11]:
!adagram-train corpus.txt out.pkl

[INFO] 2018-10-20 10:09:06,251 Building dictionary...
[INFO] 2018-10-20 10:16:23,412 Done! 23621 words.
[INFO] 2018-10-20 10:23:27,885 1.39% -8.2791 0.0247 1.1/2.0 0.15 kwords/sec
[INFO] 2018-10-20 10:23:33,166 2.77% -8.1646 0.0243 1.1/2.0 12.12 kwords/sec
[INFO] 2018-10-20 10:23:38,347 4.16% -8.0490 0.0240 1.1/3.0 12.35 kwords/sec
[INFO] 2018-10-20 10:23:43,390 5.54% -7.9348 0.0236 1.1/3.0 12.69 kwords/sec
[INFO] 2018-10-20 10:23:48,327 6.93% -7.8237 0.0233 1.2/3.0 12.96 kwords/sec
[INFO] 2018-10-20 10:23:53,121 8.31% -7.7163 0.0229 1.2/3.0 13.35 kwords/sec
[INFO] 2018-10-20 10:23:58,024 9.70% -7.6136 0.0226 1.2/4.0 13.05 kwords/sec
[INFO] 2018-10-20 10:24:02,745 11.09% -7.5165 0.0222 1.2/5.0 13.56 kwords/sec
[INFO] 2018-10-20 10:24:07,455 12.47% -7.4255 0.0219 1.3/5.0 13.59 kwords/sec
[INFO] 2018-10-20 10:24:12,169 13.86% -7.3406 0.0215 1.3/5.0 13.58 kwords/sec
[INFO] 2018-10-20 10:24:16,868 15.24% -7.2619 0.0212 1.3/5.0 13.62 kwords/sec
[INFO] 2018-10-20 10:24:21,660 16.63% -7.1891 

Обученная модель загружается вот так.

In [ ]:
vm = adagram.VectorModel("out.pkl")

In [137]:
# модель обученная на большом корпусе (острожно 1.5 гб)
# !wget 'https://s3.amazonaws.com/kostia.lopuhin/all.a010.p10.d300.w5.m100.nonorm.slim.joblib'
# vm = adagram.VectorModel.load('all.a010.p10.d300.w5.m100.nonorm.slim.joblib')

Посмотрим на значения каких-нибудь слова.

In [146]:
vm.word_sense_probs('владимир')

[(0, 0.07969982734502631),
 (1, 0.20912868267763754),
 (2, 0.010003418715142695),
 (5, 0.1790273983193883),
 (6, 0.15981042441988372),
 (7, 0.21772156406906665),
 (8, 0.1444723292321459)]

Посмотрим какие слова близки к каждому из значений.

In [144]:
vm.sense_neighbors('считать', 0)

[]

In [150]:
vm.sense_vector('считать', 1).shape

(300,)

In [147]:
vm.sense_neighbors('россия', 2)

/usr/local/lib/python3.5/dist-packages/adagram/model.py:124: RuntimeWarning: invalid value encountered in true_divide
  sim_matrix = np.dot(self.In, s_v) / self.InNorms


[('рф', 5, 0.62971777),
 ('украина', 1, 0.60423005),
 ('рф', 2, 0.58713263),
 ('руководитель', 1, 0.58310324),
 ('фракция', 0, 0.5822323),
 ('замруководителя', 0, 0.5765074),
 ('лдпр', 0, 0.56589395),
 ('александр', 1, 0.55776757),
 ('политсовет', 0, 0.54889256),
 ('вячеслав', 1, 0.54825586)]

Чтобы не гадать, можно посмотреть на все слова у которых есть хотя бы 2 устойчивых значения.

In [51]:
ambiguous = []
for word in vm.dictionary.id2word:
    probs = vm.word_sense_probs(word)
    if len(probs) > 1 and probs[0][0] < 0.8:
        ambiguous.append(word)

Как и с другими векторными моделями, проверять нужно на реальной задаче.

Возбмем определение перефразирования с прошлого семинара (и сравним сразу же)

In [65]:
corpus_xml = html.fromstring(open('paraphraser/paraphrases.xml', 'rb').read())
texts_1 = []
texts_2 = []
classes = []

for p in corpus_xml.xpath('//paraphrase'):
    texts_1.append(p.xpath('./value[@name="text_1"]/text()')[0])
    texts_2.append(p.xpath('./value[@name="text_2"]/text()')[0])
    classes.append(p.xpath('./value[@name="class"]/text()')[0])
    
data = pd.DataFrame({'text_1':texts_1, 'text_2':texts_2, 'label':classes})

Функцию, которая для текста делает векторное представление нужно немного переписать.

Что тут происходит?

In [ ]:
# vm.disambiguate('владимир', ['россия', 'путин', 'кремль', "соловьев"]).argmax()
# a = [0,1,2]
# for i in range(len(a)-1):
#     left = max(0, i-5)
#     print(a[i], a[left:i], a[i+1:i+5])

In [290]:
def get_embedding_adagram(text, model, window, dim):
    text = text.split()
    
    
    word2context = []
    for i in range(len(text)-1):
        left = max(0, i-window)
        word = text[i]
        left_context = text[left:i]
        right_context = text[i+1:i+window]
        context = left_context + right_context
        word2context.append((word, context))
    
    
    
    vectors = np.zeros((len(word2context), dim))
    
    for i,word in enumerate(word2context):
        word, context = word
        try:
            sense = model.disambiguate(word, context).argmax()
            v = model.sense_vector(word, sense)
            vectors[i] = v # просто умножаем вектор на частоту
        
        except (KeyError, ValueError):
            continue
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    
    return vector
        

In [68]:
data['text_1_norm'] = data['text_1'].apply(normalize)
data['text_2_norm'] = data['text_2'].apply(normalize)

Сделаем все точно также как и на прошлом семинаре.

In [151]:
dim = 100
X_text_1 = np.zeros((len(data['text_1_norm']), dim))
X_text_2 = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1[i] = get_embedding(text, vm, 5, dim)
    
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2[i] = get_embedding(text, vm, 5, dim)

/usr/local/lib/python3.5/dist-packages/adagram/model.py:171: RuntimeWarning: divide by zero encountered in log
  z = np.log(z)


In [152]:
X_text = np.concatenate([X_text_1, X_text_2], axis=1)

In [153]:
y = data['label'].values

In [161]:
train_X, valid_X, train_y, valid_y = train_test_split(X_text, y,random_state=1)
clf = RandomForestClassifier(n_estimators=200,
                             class_weight='balanced')
clf.fit(train_X, train_y)
preds = clf.predict(valid_X)
print(classification_report(valid_y, preds))


              precision    recall  f1-score   support

          -1       0.59      0.50      0.54       629
           0       0.48      0.79      0.59       737
           1       0.43      0.05      0.10       441

   micro avg       0.51      0.51      0.51      1807
   macro avg       0.50      0.45      0.41      1807
weighted avg       0.50      0.51      0.45      1807



Сделаем ещё нормальную валидацию.

In [158]:
train_X, valid_X, train_y, valid_y = train_test_split(X_text, y, random_state=1)
clf = LogisticRegression(C=1000, class_weight='balanced',  multi_class='auto')
clf.fit(train_X, train_y)
preds = clf.predict(valid_X)
print(classification_report(valid_y, preds))


/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

          -1       0.47      0.47      0.47       629
           0       0.46      0.45      0.46       737
           1       0.31      0.32      0.31       441

   micro avg       0.43      0.43      0.43      1807
   macro avg       0.41      0.42      0.41      1807
weighted avg       0.43      0.43      0.43      1807



In [162]:
kf = KFold(n_splits=5)
f1_scores = []
for train_index, test_index in kf.split(X_text):
    
    X_train, X_test = X_text[train_index], X_text[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    clf = RandomForestClassifier(n_estimators=200,
                             class_weight='balanced')
    
    clf.fit(X_train, y_train)
    preds = clf.predict(X_test)
    f1_scores.append(f1_score(y_test, preds, average='micro'))
    
print(np.mean(f1_scores))

0.4654768912690779


### WSD WSI

Адаграм создаваля как раз для того, чтобы находить и дизамбигуировать многозначные слова.

Посмотрим как он это делает на данных с соревнования Диалога - http://www.dialog-21.ru/evaluation/2018/disambiguation/

Возьмем один из наборов данных и запустим бейзлайновый скрипт на адаграме (тут используется большая адаграмовская модель и майстем)

In [164]:
import re
import sys
import adagram
from pymystem3 import Mystem
import pandas as pd
import numpy as np
from sklearn.metrics import adjusted_rand_score
import tqdm


mystem = Mystem()


def disambiguate(model, word, context):
    word, = lemmatized_context(word)
    probs = model.disambiguate(word, lemmatized_context(context))
    return 1 + probs.argmax()


def lemmatized_context(s):
    # This adagram model was trained with mystem lemmatizer, so better
    # use it here as well.
    return [w.lower() for w in mystem.lemmatize(s) if re.match('[\w\-]+$', w)]




In [166]:

df = pd.read_csv('train.csv', sep='\t')


df['predict_sense_id'] = [disambiguate(vm, word, context)
                          for word, context in tqdm.tqdm(zip(df['word'], df['context']), total=len(df))]

if df['gold_sense_id'].any():
    per_word = df.groupby('word').aggregate(
        lambda f: adjusted_rand_score(
            f['gold_sense_id'], f['predict_sense_id']))
    per_word_ari = per_word['predict_sense_id']
    
#     for word, ari in zip(per_word.index, per_word_ari):
#             print('{:<20} {:+.4f}'.format(word, ari))
    print('Mean word ARI: {:.4f}'.format(np.mean(per_word_ari)))




  0%|          | 0/2073 [00:00<?, ?it/s]/usr/local/lib/python3.5/dist-packages/adagram/model.py:171: RuntimeWarning: divide by zero encountered in log
  z = np.log(z)
100%|██████████| 2073/2073 [00:01<00:00, 1164.69it/s]


Mean word ARI: 0.1789


В качестве метрики используется Adjuster Rand Index. https://en.wikipedia.org/wiki/Rand_index , http://scikit-learn.org/stable/modules/generated/sklearn.metrics.adjusted_rand_score.html

In [174]:
df = pd.read_csv('train.csv', sep='\t')

In [176]:
df.head(20)

,context_id,word,gold_sense_id,predict_sense_id,positions,context
0,1,дар,1,NaN,18-22,Отвергнуть щедрый дар
1,2,дар,1,NaN,21-28,покупать преданность дарами и наградами
2,3,дар,1,NaN,19-23,Вот яд – последний дар моей Изоры
3,4,дар,1,NaN,81-87,Основная функция корильных песен – повеселить ...
4,5,дар,1,NaN,151-157,Но недели две спустя (Алевтина его когда-то об...
5,6,дар,1,NaN,95-99,Мать Ревекки приберегала кусок и на праздник п...
6,7,дар,1,NaN,205-210,Время от времени Лидия Михайловна «доставала» ...
7,8,дар,1,NaN,80-84,Недавно приезжавший в Оргеев посол РФ в Молдав...
8,9,дар,2.1,NaN,18-22,Жизнь – бесценный дар
9,10,дар,2.1,NaN,25-29,Такая любовь – настоящий дар судьбы


Попробуем решить эту задачу фастекстом.

In [186]:
def get_embedding(text, model, dim):
    text = text.split()
    
    # чтобы не доставать одно слово несколько раз
    # сделаем счетчик, а потом векторы домножим на частоту
    words = Counter(text)
    total = len(text)
    vectors = np.zeros((len(words), dim))
    
    for i,word in enumerate(words):
        try:
            v = model[word]
            vectors[i] = v*(words[word]/total) # просто умножаем вектор на частоту
        except (KeyError, ValueError):
            continue
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    
    return vector
        

In [178]:
corpus = [text.split() for text in data_rt['content'].apply(tokenize)]
fast_text = gensim.models.FastText(corpus, size=200, max_vocab_size=100000)

In [261]:
%%time
# corpus = [tokenize(text).split() for text in open('sentences_100k_wiki.txt')]
fast_text = gensim.models.FastText(corpus, size=200, max_vocab_size=100000)

CPU times: user 14min 56s, sys: 1.44 s, total: 14min 58s
Wall time: 5min 10s


In [262]:
fast_text.save('model_100k_wiki')

In [200]:
grouped_df = df.groupby('word')[['word', 'context', 'gold_sense_id']]

In [190]:

for key, _ in grouped_df:
    print(grouped_df.get_group(key), "\n\n")

   word                                            context
0   дар                              Отвергнуть щедрый дар
1   дар            покупать преданность дарами и наградами
2   дар                  Вот яд – последний дар моей Изоры
3   дар  Основная функция корильных песен – повеселить ...
4   дар  Но недели две спустя (Алевтина его когда-то об...
5   дар  Мать Ревекки приберегала кусок и на праздник п...
6   дар  Время от времени Лидия Михайловна «доставала» ...
7   дар  Недавно приезжавший в Оргеев посол РФ в Молдав...
8   дар                              Жизнь – бесценный дар
9   дар                Такая любовь – настоящий дар судьбы
10  дар                             Неисчислимы Божии дары
11  дар  Дар напрасный, дар случайный, / Жизнь, зачем т...
12  дар  Господа, – воскликнул я вдруг от всего сердца,...
13  дар  Слишком многого она ожидала, слишком многого т...
14  дар  То, что выглядит как наша жертва Ему, в высшей...
15  дар  Смерть любимых, дорогих людей – это дар, котор.

2019  знак   Номерной знак вашей машины, случаем, не ВКР-821? 


       word                                            context
2020  знамя  Над ополчением, вышедшим к Москве, развевалось...
2021  знамя  Предстоит решить, кто понесет знамя нашей стра...
2022  знамя  Двести захваченных фашистских знамен и штандар...
2023  знамя  Развернулись, защелкав на ветру, полковое и ба...
2024  знамя  В сорок втором году в окружении осталось знамя...
2025  знамя  – Скоро под это знамя придут другие солдаты, –...
2026  знамя  Дежурный Давилин принес из угла комнаты древко...
2027  знамя  Постаревший отставной военный […] размахивал к...
2028  знамя  Большой кабинет. В углу много знамен. На одной...
2029  знамя                                    Знамя гуманизма
2030  знамя                        поднять знамя борьбы за мир
2031  знамя                        встать под знамя демократии
2032  знамя  Монгольский народ строит свое светлое будущее ...
2033  знамя  Я верю в дальновидность советских литера

In [208]:
grouped_df.get_group(key)

TypeError: 'DataFrame' objects are mutable, thus they cannot be hashed

In [196]:
from sklearn.cluster import *

In [199]:
grouped_df.get_group(key)['context']

TypeError: 'DataFrame' objects are mutable, thus they cannot be hashed

In [288]:
ARI = []

for key, _ in grouped_df:
    texts = grouped_df.get_group(key)['context'].apply(tokenize)
    X = np.zeros((len(texts), 200))
    
    for i, text in enumerate(texts):
        X[i] = get_embedding(text, fast_text, 200)
        
    cluster = SpectralClustering(n_clusters=)
    cluster.fit(X)
    labels = np.array(cluster.labels_)+1
    
    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
    
#     print(key, '  ', adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))
print(np.mean(ARI))

0.013128628041642358


In [250]:
grouped_df.get_group(key)['gold_sense_id']

2064    1
2065    1
2066    1
2067    1
2068    1
2069    2
2070    2
2071    2
2072    2
Name: gold_sense_id, dtype: object

In [251]:
labels

array([3, 3, 3, 3, 3, 1, 2, 3, 3])

In [252]:
adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels)

0.18181818181818177

In [244]:
text

'я выскочила из исторического музея в летнее кафе под большими зонтами'

In [217]:
cluster.labels_

array([-1, -1, -1, -1,  0,  0,  0, -1, -1, -1, -1, -1,  0, -1, -1, -1,  0,
       -1, -1, -1, -1, -1, -1, -1, -1,  0,  0,  0, -1, -1, -1, -1,  0, -1,
        0,  0])